In [8]:
import numpy as np
import pandas as pd
file_path = "/Users/conny/Library/CloudStorage/OneDrive-EmoryUniversity/computational_legislature/UK/Bills/Bills_Latest_Stage/Bills.csv"
temp = pd.read_csv(file_path)
#Whenver there is a NaN in billId, replace it with the previous billId
temp['billId'] = temp['billId'].replace(np.nan, method='ffill')
#Get the second column of temp['billId'] into a list called billId
billId_list = temp['billId'].tolist()


print("The number of bills is: ", len(billId_list))

The number of bills is:  3268


In [ ]:
import requests
import time
import json
import csv

# Base URL for the "History" endpoint
BASE_URL = "https://bills-api.parliament.uk/api/v1/Bills/"

# Store the results in a list
results = []

def save_data(data, filename="data.json"):
    """Save the data to a JSON file."""
    with open(filename, "w") as f:
        json.dump(data, f)

for i in range(len(billId_list)):
    billId = billId_list[i]
    # Construct the URL for the API endpoint
    url = f"{BASE_URL}{billId}/Publications"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # This will throw an exception if the HTTP code is 4xx or 5xx

        data = response.json()
        results.append(data)

        # Save the data every 100 requests
        if billId % 100 == 0:
            save_data(results, filename=f"data_up_to_{billId}.json")

        # Sleep for 5 seconds between requests to avoid rate limits
        time.sleep(0.1)

    except requests.RequestException as e:
        print(f"Error fetching data for ID {billId}: {e}")
        # Optionally, sleep for longer if an error occurs to give the server a break
        time.sleep(0.1)

# Save the final batch of data
save_data(results, filename=f"Publication.json")

#I want to create a csv file where the first 2 columns are billId and stageId
# , and the third column is the amendment information from the json file



In [7]:
import json
import requests
import csv
import pandas as pd
import numpy as np
import time
def get(BillID):
    #These indicators are not used in this current version of script to extract member data
    #Indicator for whether the request is complete/ Exit condition out of the while loop
    isComplete = False

    #Specify the starting point for th next chunk of data in each new request
    offset = 0

    #Iteration number
    iteration = 1

    #Total number of entries
    total_entries = 0

    #Store the error ID, indicating that the response is not found
    errors_404 = 0

    #Store the error ID, indicating there exists an internal server error
    errors_500 = 0

    #Store the error list, indicating that the request is bad
    errors_400 = 0

    #Appending each response to this results list
    results = []

    url = f"https://bills-api.parliament.uk/api/v1/Bills/{BillID}/Publications"
    response = requests.get(url)

    #Currently, the success code is 200 from "https://bills-api.parliament.uk/index.html"
    if response.status_code == 200:

        #Parse the JSON response into a Python dictionary
        data = response.json()

        #Append the data to the results list
        results.append(data)

    elif response.status_code == 404:
        #Append the error message to the errors list
        errors_404 = BillID

        print(f"Error fetching data for ID as it is not found {BillID}")
        # Optionally, sleep for longer if an error occurs to give the server a break
        # As far as I know, the server does not have a rate limit
        time.sleep(1)
    
    elif response.status_code == 500:
        errors_500 = BillID

        print(f"Error fetching data for ID due to server error {BillID}")

    elif response.status_code == 400:
        errors_400 = BillID

        print(f"Error fetching data for ID due to bad request {BillID}")

    return results, errors_404, errors_500, errors_400



billId_list = [1000,2001]
final = []
final_nopub = []
final_404 = []
final_500 = []
final_400 = []
for i in range(len(billId_list)):
    BillID = billId_list[i]
    bills, errors_404, errors_500, errors_400 = get(BillID)
    if errors_404 != 0:
        final_404.append(errors_404)
    elif errors_500 != 0:
        final_500.append(errors_500)
    elif errors_400 != 0:
            final_400.append(errors_400)
    elif pd.json_normalize(bills) is None:
            final_nopub.append(BillID)
    elif bills is not None:
        norm = pd.json_normalize(bills, record_path=['publications'], meta=['billId'])
        final.append(norm)

final_df = pd.concat(final, ignore_index=True)
